In [1]:
import os
import csv
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt 
import matplotlib
from matplotlib.animation import FuncAnimation
from matplotlib.animation import writers

In [2]:
def read_data(path):
    S = []
    costs = []
    first = True
    with open(path, "r") as f:
        csv.reader(f, delimiter=" ")
        
        for line in f:
            info = line.split(" ")
            if first:
                first = False
                n_sets = info[0]
                n_elems = info[1]
            else:
                s = set()
                costs.append(int(info[0]))
                info.pop(0)
                for e in info:
                    e.replace("\n", "")
                    s.add(int(e))
                S.append(s)
    return int(n_elems), int(n_sets), S, costs

In [3]:
def s_to_matrix(n_elems, n_sets, S):
    
    # Inicializar matriz todo ceros
    matrix = np.zeros((n_sets, n_elems))
    
    # Los elementos van de 1 a n_elems
    
    col = 0
    for s in S:
        for e in s:
            matrix[e - 1][col] = 1
        col += 1
        
    return matrix
        

# <center> ACO for Set Cover (Ant Cover)</center>

In [4]:
# Terminación: cuando todos los elementos están cubiertos
def construct_termination(n_elems, n_currently_covered):
    return n_elems == n_currently_covered

In [5]:
# Coger un conjunto j es factible si cubre el elemento i e i no está ya cubierto
def isFeasible(j, i, matrix, currently_uncovered):
    return (matrix[i][j] == 1) and (i in currently_uncovered)

In [6]:
# Calcula cuántos elementos nuevos cubre un conjunto
def calculate_n_new_covered(n_elems, matrix, j, currently_uncovered):
    n_new_covered = 0
    for i in range(n_elems):
        # Si el conjunto j cubre el elemento i, y el elemento i no está ya cubierto, se suma uno
        if (matrix[i][j] == 1) and (i in currently_uncovered):
            n_new_covered += 1
    return n_new_covered

In [7]:
def calculate_probability(i, j, costs, pheromone_trail, matrix, n_elems, n_sets, currently_uncovered):
    # psi_j = número de nuevos elementos que puede cubrir el conjunto j
    psi_j = calculate_n_new_covered(n_elems, matrix, j, currently_uncovered)
    
    # eta = heurística
    eta = psi_j / costs[j]
    
    # Importancia relativa de la heurística con respecto a la feromona
    beta = 0.5 # >= 0
    
    numerator = pheromone_trail[j] * pow(eta, beta)
    
    denominator = 0
    
    # Para cada conjunto q que cubre el elemento i
    for q in range(n_sets):

        if isFeasible(i, j, matrix, currently_uncovered):
            psi_q = calculate_n_new_covered(n_elems, matrix, q, currently_uncovered)
            denominator += pheromone_trail[j] * (psi_q / costs[q])
    
    return numerator / denominator

In [8]:
def updateBestSolution(sol, best_cost, best_solution, evaporation_rate, costs, epsilon):
    # Actualizar nueva mejor solución
    total_cost = 0
    for j in sol:
        total_cost += costs[j]
    
    if total_cost < best_cost:
        best_cost = total_cost
        best_solution = sol
    
        # Actualizar phmin y phmax
        phmax = 1 / ((1 - evaporation_rate) * best_cost)
        phmin = epsilon*phmax
        
    return best_solution, phmax, phmin

In [9]:
# Devuelve los elementos que cubre el conjunto s
def covered_by(s, n_elems, matrix):
    elems = []
    for elem in range(n_elems):
        if matrix[elem][s] == 1:
            elems.append(elem)
    return elems

In [10]:
# Una hormiga construye una solución
# Empieza con una solución vacía solution_construction
# Se añaden conjuntos iterativamente a solution_construction hasta que todos los elementos están cubiertos
# Se elige un elemento que debe ser cubierto (orden: aleatorio, natural o ascendiente por coste)
# Se elige el conjunto que lo cubre basándose en una función de probabilidad (feromonas)
def constructAntSolution(matrix, n_sets, n_elems, pheromone_trail):
    solution_construction = []
    currently_covered = []
    currently_uncovered = range(n_elems)
    
    while not construct_termination(n_elems, len(currently_covered)):
        p = []
        a = []
        for i in range(n_elems): # Elegir un elemento a cubrir
            for j in range(n_sets): # Elegir un conjunto que lo cubra
                if isFeasible(i, j, matrix, currently_uncovered):
                    # Calcular la probabilidad de elegir el conjunto j
                    p.append(calculate_probability(i, j, costs, pheromone_trail, matrix, n_elems, n_sets, currently_uncovered))
                    a.append(j)
        
        chosen_set = np.random.choice(a, 1, p=p)[0]
        
        # Actualizar elementos cubiertos y no cubiertos
        for elem in covered_by(chosen_set):   
            currently_covered.append(elem)
            currently_uncovered.remove(elem)
        
        # Añadir el conjunto seleccionado a la solución
        solution_construction.append(chosen_set)

    return solution_construction

In [11]:
def updateBestSolution(sol, best_cost, best_solution, evaporation_rate, costs, epsilon):
    # Actualizar nueva mejor solución
    total_cost = 0
    for j in sol:
        total_cost += costs[j]
    
    if total_cost < best_cost:
        best_cost = total_cost
        best_solution = sol
    
        # Actualizar phmin y phmax
        phmax = 1 / ((1 - evaporation_rate) * best_cost)
        phmin = epsilon*phmax
        
    return best_solution, phmax, phmin

In [12]:
def updatePheromone(pheromone_trail, n_sets, best_solution, costs, phmax, phmin, evaporation_rate):
    
    # Evaporation
    for j in range(n_sets):
        pheromone_trail[j] *= (1 - evaporation_rate)
    
    # Para cada conjunto de la mejor solución, se le suma el valor de la feromona (limitado por phmax y phmin)
    for q in best_solution:
        phq = pheromone_trail[q] + (1 / costs[q])
        if phq > phmax:
            pheromone_trail[q] = phmax
        elif phq < phmin:
            pheromone_trail[q] = phmin
        else:
            pheromone_trail[q] = phq

In [13]:
def aco(matrix, m, n_elems, n_sets, tours, costs):  
    pheromone_trail = []
    
    best_solution = []
    best_cost = math.inf

    epsilon = 0.2 # 0 < epsilon < 1 coeficiente de relación
    phmax = math.inf
    phmin = math.inf
    
    evaporation_rate = 0.3
    
    # Initialize trail
    pheromone_trail = [0.000000001]*n_sets

    for t in range(tours):
        solutions = []
       
        # Cada hormiga construye una solución, que se guarda en solutions
        for ant in range(m):
            
            sol = constructAntSolution(matrix, n_sets, n_elems, pheromone_trail)
            print(sol)
            solutions.append(sol)

        # Actualizar sendero de feromonas
        updatePheromone(pheromone_trail, n_sets, best_solution, costs, phmax, phmin, evaporation_rate)
        
        # Actualizar mejor solución
        updateBestSolution(sol, best_cost, best_solution, evaporation_rate, costs, epsilon)

In [14]:
path = os.path.join(os.getcwd(), 'dataset', 'scp', "set_cover_100.txt")
n_elems, n_sets, S, costs = read_data(path)
matrix = s_to_matrix(n_sets, n_elems, S)

m = 20 # number of ants

tours = 50 # number of tours
iterations = 1


for it in range(iterations):
    
    #start=datetime.now()
    
    aco(matrix, m, n_elems, n_sets, tours, costs)
    
    #time = datetime.now()-start

    m+=5

UsageError: Line magic function `%%debug` not found.
